<a href="https://colab.research.google.com/github/rajeshsahu09/CS69002_9A_18CS60R19/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

# Run on **GPU**



### Import Header Files

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import time

### Load the Dataset and Visualise

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
# df_train.head()
url = "https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv"
df = pd.read_csv(url, sep='\t')

In [3]:
df.shape

(17999, 2)

In [0]:
df_1 = df[df['label']==1] # get only label == 1
df_0 = df[df['label']==0] # get only label == 0

range_1 = int(0.9*len(df_1))
range_2 = int(0.9*len(df_0))

np.random.seed(42) # fix the seed value
df_1 = df_1.iloc[np.random.permutation(len(df_1))] # shuffled the data set of label==1
df_0 = df_0.iloc[np.random.permutation(len(df_0))] # shuffled the data set of label==0

temp_1_train = df_1.iloc[:range_1]
temp_2_train = df_0.iloc[:range_2]
df_train = pd.concat([temp_1_train, temp_2_train])

temp_1_val = df_1.iloc[range_1:]
temp_2_val = df_0.iloc[range_2:]
df_val = pd.concat([temp_1_val, temp_2_val])

In [5]:
df_train = df_train.iloc[np.random.permutation(len(df_train))] # shuffled the train data set
df_val = df_val.iloc[np.random.permutation(len(df_val))] # shuffled the validation data set
len(df_train), len(df_val)

(16198, 1801)

In [6]:
print('Number of Negative movie reviews', len(df_train[df_train['label']==0]))
print('Number of Positive movie reviews', len(df_train[df_train['label']==1]))
print('Number of movie reviews', len(df_train['label']))

Number of Negative movie reviews 8094
Number of Positive movie reviews 8104
Number of movie reviews 16198


In [7]:
print('Number of Negative movie reviews', len(df_val[df_val['label']==0]))
print('Number of Positive movie reviews', len(df_val[df_val['label']==1]))
print('Number of movie reviews', len(df_val['label']))

Number of Negative movie reviews 900
Number of Positive movie reviews 901
Number of movie reviews 1801


In [0]:
url = "https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv"
df_test = pd.read_csv(url, sep='\t')

## Data pre-processing

### Get Train Data

In [0]:
train_raw_text_reviews = df_train['text'].astype(str).tolist()
train_text_labels = df_train['label'].astype(int).tolist()

###Get Validation Data

In [0]:
val_raw_text_reviews = df_val['text'].astype(str).tolist()
val_text_labels = df_val['label'].astype(int).tolist()

###Get Test Data

In [0]:
test_raw_text_reviews = df_test['text'].astype(str).tolist()
test_text_labels = df_test['label'].astype(int).tolist()

#### Cleaning the raw input data

In [12]:
import re
import string
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

# Finding stop words
stop_words = set(stopwords.words('english'))

# import spacy
# print('spaCy Version: %s' % (spacy.__version__))
# spacy_nlp = spacy.load('en_core_web_sm')
# # stop word list
# stop_words = set(spacy.lang.en.stop_words.STOP_WORDS)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def preprocess_document(doc):
    # negative sense should not be eleminated + some short representation
    CONTRACTIONS = {"didn't":"did not", "mayn't":"may not", "can't":"can not", "won't":"will not", "isn't":"is not", "amn't":"am not",\
                  "aren't":"are not", "wasn't":"was not", "weren't":"were not", "couldn't":"could not", \
                  "wouldn't":"would not", "don't":"do not", "doesn't":"does not",\
                  "i'll":"i will", "you'll":"you will", "they'll":"they will",\
                  "may've":"may have", "can've":"can have", "will've":"will have", "you've":"you have", \
                  "could've":"could have", "would've":"would have", "you've":"you have", "they":"they have",\
                  "i've":"i have", "you've":"you have", "we've":"we have", "there's":"there is", "i'm":"i am",\
                  "it's":"it is", "what's":"what is", "where's":"where is", "how's":"how is", "i'd":"i had"}
    punctuation = string.punctuation + "\n\n"
    punc_replace = ''.join([' ' for s in punctuation]) # required for replacing punctuation with null ('')
    doc_clean = doc.replace('-', ' ') # replace - with null str
    doc_clean = (doc_clean.encode('ascii', 'ignore')).decode("utf-8")
    doc_clean = doc_clean.replace('<br />', '') # replace <br /> with ''
    doc_clean = doc_clean.replace("’", "'") # replace <br /> with null str
    doc_clean = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in doc_clean.split(' ')] # replacing some common short forms
    doc_clean = " ".join(doc_clean) # list to sentence of strings
    doc_clean = re.sub(r'\W +', ' ', doc_clean) # except [a-zA-Z0-9_]
    doc_clean = re.sub(r'\d+', ' ', doc_clean) # remove numbers [0-9]
    trans_table = str.maketrans(punctuation, punc_replace); # replace punctuations with ' '
    doc_clean = ' '.join([word.translate(trans_table) for word in doc_clean.split(' ')])
    doc_clean = doc_clean.split(' ')
    doc_clean = [word for word in doc_clean if len(word) > 1]
    # removing the stopwords from a sentence
    doc_clean = [word for word in doc_clean if not word.lower() in stop_words or word.lower() == 'not' or word.lower() == 'no']
    return doc_clean

In [14]:
train_text_reviews = [preprocess_document(review.lower()) for review in train_raw_text_reviews]
print (train_text_reviews[len(train_text_reviews)-2])
print (train_text_labels[len(train_text_labels)-2])

['three', 'tales', 'told', 'film', 'seemed', 'shot', 'without', 'knowledge', 'combined', 'vignette', 'film', 'makers', 'relate', 'three', 'vignettes', 'connected', 'shrink', 'martin', 'kove', 'although', 'never', 'see', 'leads', 'kove', 'first', 'vignette', 'sexy', 'vivian', 'schilling', 'woman', 'afraid', 'everything', 'sun', 'makes', 'adrian', 'monk', 'look', 'brave', 'paranoia', 'laced', 'evening', 'home', 'alone', 'literally', 'scream', 'vivian', 'ridiculous', 'things', 'spends', 'majority', 'time', 'nighty', 'shows', 'amazing', 'features', 'film', 'worst', 'not', 'nail', 'biting', 'second', 'vignette', 'owned', 'bill', 'paxton', 'portrays', 'roommate', 'hell', 'geeky', 'roommate', 'allows', 'take', 'complete', 'advantage', 'bill', 'whenever', 'last', 'vignette', 'funny', 'man', 'fears', 'death', 'take', 'moment', 'much', 'like', 'pal', 'choked', 'death', 'olive', 'not', 'interesting', 'movie', 'whole', 'seems', 'chopped', 'together', 'little', 'thought', 'involved', 'must', 'bill'

In [15]:
val_text_reviews = [preprocess_document(review.lower()) for review in val_raw_text_reviews]
print (val_text_reviews[len(val_text_reviews)-2])
print (val_text_labels[len(val_text_labels)-2])

['even', 'one', 'not', 'realize', 'sellers', 'poor', 'health', 'time', 'filming', 'passed', 'away', 'film', 'release', 'fiendish', 'plot', 'dr', 'fu', 'manchu', 'would', 'painful', 'viewing', 'supposedly', 'lampoon', 'sax', 'rohmer', 'famous', 'oriental', 'villain', 'lacks', 'focus', 'potential', 'satirical', 'commentary', 'anti', 'oriental', 'overtones', 'rohmer', 'concept', 'ignored', 'indeed', 'movie', 'employs', 'racist', 'insults', 'hardly', 'actual', 'jokes', 'gags', 'mostly', 'actors', 'behaving', 'idiotically', 'spouting', 'dreary', 'lines', 'especially', 'distressing', 'see', 'sid', 'caesar', 'forced', 'spout', 'curses', 'racial', 'slurs', 'attempted', 'laughs', 'actors', 'embarrass', 'well', 'peter', 'sellers', 'plays', 'dual', 'roles', 'sinister', 'fu', 'manchu', 'trying', 'concoct', 'formula', 'regain', 'youth', 'stalwart', 'british', 'foe', 'nayland', 'smith', 'sellers', 'not', 'one', 'hundred', 'per', 'cent', 'bad', 'conveys', 'quirky', 'warmth', 'smith', 'discusses', 'fe

In [16]:
test_text_reviews = [preprocess_document(review.lower()) for review in test_raw_text_reviews]
print (test_text_reviews[len(test_text_reviews)-2])
print (test_text_labels[len(test_text_labels)-2])

['shining', 'know', 'weird', 'movie', 'movie', 'everyone', 'people', 'claim', 'not', 'like', 'horror', 'films', 'always', 'say', 'shining', 'terrific', 'film', 'stanley', 'kubrick', 'classic', 'vision', 'stephen', 'king', 'horror', 'tale', 'madness', 'blood', 'incredible', 'film', 'wither', 'seen', 'not', 'heard', 'know', 'lines', 'know', 'classic', 'images', 'could', 'forget', 'jack', 'johnny', 'could', 'forget', 'work', 'no', 'play', 'make', 'jack', 'dull', 'boy', 'could', 'forget', 'chilling', 'ending', 'film', 'unforgettable', 'honestly', 'opinion', 'kubrick', 'best', 'work', 'know', 'lot', 'argument', 'department', 'lot', 'people', 'say', 'space', 'odyssey', 'clockwork', 'orange', 'even', 'dr', 'strangelove', 'film', 'pioneered', 'film', 'making', 'shining', 'perfected', 'tale', 'isolation', 'madness', 'terrifying', 'images', 'ultimate', 'ghost', 'story', 'crawl', 'underneath', 'skin', 'jack', 'torrance', 'jack', 'son', 'danny', 'jack', 'wife', 'wendy', 'arrive', 'overlook', 'hote

In [17]:
import gc
gc.collect()

0

In [0]:
def build_vocabulary(sentences):
    # Build vocabulary
    dictWordCount = {}
    for sent in sentences:
        for word in sent:
            dictWordCount[word] = 0 # initialising the dict value to zero
    for sent in sentences:
        for word in sent:
            dictWordCount[word] += 1 # updating the dictionary count
    
    temp = dictWordCount.copy()
    for key, val in temp.items():
        if(dictWordCount[key] <= 5):
            del dictWordCount[key]
    
    # Mapping from index to word
    vocabulary_inv = sorted(dictWordCount, key=dictWordCount.__getitem__, reverse=True)
    
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return vocabulary, vocabulary_inv

###Creating Tokens and Types

In [0]:
word_to_ix, ix_to_word = build_vocabulary(train_text_reviews+val_text_reviews+test_text_reviews)

In [20]:
VOCAB_SIZE = len(word_to_ix)
ix_to_word[word_to_ix['kick']]=='kick', word_to_ix['kick'], VOCAB_SIZE

(True, 1814, 25047)

In [21]:
gc.collect()

403

## PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable # Automatic gradients are calculated and back-propagated through the computational graph
import copy
import csv
import time
from google.colab import files

## Generate the BOW Vectors

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix)).to('cuda:0') #, device=device) # make 1D vector of len = vocab size
    for word in sentence:
        if word not in word_to_ix:
#             raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            pass
        else:
            vec[word_to_ix[word]]+=1 # count the number of occurance of same word in a sentences
            
    return vec.view(1, -1)

### Prepare Input Train and Validation torch vector

In [24]:
# store the bag of word vectors for each sentences and wraping to tensor of torch type
tic = time.time()
train_data = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in train_text_reviews]
num_train_data = len(train_text_reviews)

val_data = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in val_text_reviews]
num_val_data = len(val_text_reviews)
toc = time.time()
num_train_data, num_val_data, (toc-tic)

(16198, 1801, 94.47736620903015)

### Prepare Test Instance

In [0]:
data_test = []
data_test = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in test_text_reviews]

### Validation Accuracy Computation

In [0]:
def evaluate_validation_accuracy(data, net):
    sum_loss = 0
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for i, instance in enumerate(data):
        label = val_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label
        
#         vec = Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') # wrap to tensor of torch type for instance
        
#         prob = net.forward(vec) # forward pass

        prob = net.forward(instance)
    
        _class = 1 if prob.item() > 0.5 else 0 # sigmoid activated
        
        loss = loss_function(prob, label) # compute the loss
        
        sum_loss += float(loss.item())
        
#         if(int(label) == _class and _class == 1):
#             true_positive += 1
            
#         if(int(label) == _class and _class == 0):
#             true_negative += 1
        
#         if(_class == 1 and int(label) == 0):
#             false_positive += 1
            
#         if(_class == 0 and int(label) == 1):
#             false_negative += 1
    
    
#     precision = float(true_positive) / (true_positive + false_positive)
#     recall = float(true_positive) / (true_positive + false_negative)
#     f_score = float(2)*precision*recall / (precision + recall)
#     accuracy = float(1)*(true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)
    
    return float(sum_loss)/len(data)#, float(100)*accuracy, precision, recall, f_score

##Task-1

In [0]:
task1 = [(50, 'Task1A.mdl'), (100, 'Task1B.mdl'), (200, 'Task1C.mdl'), (500, 'Task1D.mdl')]

In [0]:
for num_of_hidden, task_name in task1:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.hidden_size = hidden_size
            self.i2h = nn.Linear(input_size, hidden_size) # initialises weights and biases i2h
            self.h2o = nn.Linear(hidden_size, output_size) # initialises weights and biases h2o

        def forward(self, x):
            x = torch.relu(self.i2h(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden, num_of_output).to('cuda:0') # initialises weights and biases
    
    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)
    
    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)
        
        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))
    
    torch.save(bow, task_name) ## Save the model
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))
    
    print ('\n')

Epoch 1/100
[On Training] ==> Time: 26.26s, Train Loss: 0.679907998, Train Accuracy: 63.10%
[On Validation] ==> Val loss: 0.662730204, Total Time: 26.26s
Epoch 2/100
[On Training] ==> Time: 24.77s, Train Loss: 0.641387382, Train Accuracy: 73.63%
[On Validation] ==> Val loss: 0.617228091, Total Time: 51.03s
Epoch 3/100
[On Training] ==> Time: 24.75s, Train Loss: 0.592229105, Train Accuracy: 77.21%
[On Validation] ==> Val loss: 0.566256706, Total Time: 75.79s
Epoch 4/100
[On Training] ==> Time: 24.73s, Train Loss: 0.540704002, Train Accuracy: 79.55%
[On Validation] ==> Val loss: 0.518085913, Total Time: 100.52s
Epoch 5/100
[On Training] ==> Time: 24.76s, Train Loss: 0.494395833, Train Accuracy: 81.34%
[On Validation] ==> Val loss: 0.478558819, Total Time: 125.28s
Epoch 6/100
[On Training] ==> Time: 24.85s, Train Loss: 0.456540306, Train Accuracy: 82.70%
[On Validation] ==> Val loss: 0.448150814, Total Time: 150.13s
Epoch 7/100
[On Training] ==> Time: 25.98s, Train Loss: 0.426290917, Trai

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task1A.mdl
[On Testing] Time: 2.378115653991699, Test Accuracy: 88.60%


Epoch 1/100
[On Training] ==> Time: 26.83s, Train Loss: 0.681274268, Train Accuracy: 64.59%
[On Validation] ==> Val loss: 0.664423299, Total Time: 26.83s
Epoch 2/100
[On Training] ==> Time: 26.84s, Train Loss: 0.643418346, Train Accuracy: 74.10%
[On Validation] ==> Val loss: 0.619357091, Total Time: 53.67s
Epoch 3/100
[On Training] ==> Time: 26.83s, Train Loss: 0.594328215, Train Accuracy: 77.30%
[On Validation] ==> Val loss: 0.568000911, Total Time: 80.50s
Epoch 4/100
[On Training] ==> Time: 26.83s, Train Loss: 0.542414091, Train Accuracy: 79.57%
[On Validation] ==> Val loss: 0.519250212, Total Time: 107.33s
Epoch 5/100
[On Training] ==> Time: 26.89s, Train Loss: 0.495759835, Train Accuracy: 81.33%
[On Validation] ==> Val loss: 0.479240916, Total Time: 134.22s
Epoch 6/100
[On Training] ==> Time: 27.01s, Train Loss: 0.457674535, Train Accuracy: 82.60%
[On Validation] ==> Val loss: 0.4

In [0]:
files.download('Task1A.mdl')
files.download('Task1B.mdl')
files.download('Task1C.mdl')
files.download('Task1D.mdl')

##Task-2

In [0]:
task2 = [(10, 10, 'Task2A.mdl'), (20, 10, 'Task2B.mdl'), (30, 30, 'Task2C.mdl'), (50, 50, 'Task2D.mdl'), (100, 50, 'Task2E.mdl')]

In [0]:
for num_of_hidden1, num_of_hidden2, task_name in task2:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.i2h = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h
            self.h2h = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h2h
            self.h2o = nn.Linear(hidden_size2, output_size) # initialises weights and biases h2o

        def forward(self, x):
            x = torch.relu(self.i2h(x)) # relu activation @ hidden layer
            x = torch.relu(self.h2h(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)

    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))
    
    print("\n")

Epoch 1/100
[On Training] ==> Time: 26.33s, Train Loss: 0.696559569, Train Accuracy: 49.97%
[On Validation] ==> Val loss: 0.692734756, Total Time: 26.33s
Epoch 2/100
[On Training] ==> Time: 26.25s, Train Loss: 0.688986655, Train Accuracy: 49.97%
[On Validation] ==> Val loss: 0.684673949, Total Time: 52.58s
Epoch 3/100
[On Training] ==> Time: 26.17s, Train Loss: 0.678799666, Train Accuracy: 52.53%
[On Validation] ==> Val loss: 0.671896277, Total Time: 78.75s
Epoch 4/100
[On Training] ==> Time: 26.20s, Train Loss: 0.662443024, Train Accuracy: 68.51%
[On Validation] ==> Val loss: 0.651179718, Total Time: 104.95s
Epoch 5/100
[On Training] ==> Time: 26.07s, Train Loss: 0.635187637, Train Accuracy: 76.31%
[On Validation] ==> Val loss: 0.615992655, Total Time: 131.02s
Epoch 6/100
[On Training] ==> Time: 26.16s, Train Loss: 0.590748855, Train Accuracy: 79.36%
[On Validation] ==> Val loss: 0.562680017, Total Time: 157.18s
Epoch 7/100
[On Training] ==> Time: 26.16s, Train Loss: 0.530256079, Trai

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task2A.mdl
[On Testing] Time: 2.347750663757324, Test Accuracy: 88.34%


Epoch 1/100
[On Training] ==> Time: 26.53s, Train Loss: 0.697163920, Train Accuracy: 49.97%
[On Validation] ==> Val loss: 0.692895690, Total Time: 26.53s
Epoch 2/100
[On Training] ==> Time: 25.94s, Train Loss: 0.688107316, Train Accuracy: 50.91%
[On Validation] ==> Val loss: 0.682609272, Total Time: 52.47s
Epoch 3/100
[On Training] ==> Time: 26.39s, Train Loss: 0.676529645, Train Accuracy: 64.41%
[On Validation] ==> Val loss: 0.669844118, Total Time: 78.86s
Epoch 4/100
[On Training] ==> Time: 26.25s, Train Loss: 0.661661082, Train Accuracy: 76.71%
[On Validation] ==> Val loss: 0.652425888, Total Time: 105.11s
Epoch 5/100
[On Training] ==> Time: 25.98s, Train Loss: 0.640017168, Train Accuracy: 77.10%
[On Validation] ==> Val loss: 0.625674039, Total Time: 131.10s
Epoch 6/100
[On Training] ==> Time: 26.47s, Train Loss: 0.601781989, Train Accuracy: 78.69%
[On Validation] ==> Val loss: 0.5

In [0]:
files.download('Task2A.mdl')
files.download('Task2B.mdl')
files.download('Task2C.mdl')
files.download('Task2D.mdl')
files.download('Task2E.mdl')

##Task-3

In [0]:
task3 = [(100, 50, 10, 'Task3A.mdl'), (200, 100, 10, 'Task3B.mdl')]

In [0]:
for num_of_hidden1, num_of_hidden2, num_of_hidden3, task_name in task3:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
            self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
            self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
            self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

        def forward(self, x):
            x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
            x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
            x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))
    
    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 34.33s, Train Loss: 0.692630532, Train Accuracy: 50.27%
[On Validation] ==> Val loss: 0.691402558, Total Time: 34.33s
Epoch 2/100
[On Training] ==> Time: 33.96s, Train Loss: 0.690193089, Train Accuracy: 56.99%
[On Validation] ==> Val loss: 0.688726793, Total Time: 68.29s
Epoch 3/100
[On Training] ==> Time: 33.90s, Train Loss: 0.686891386, Train Accuracy: 65.47%
[On Validation] ==> Val loss: 0.684602229, Total Time: 102.19s
Epoch 4/100
[On Training] ==> Time: 33.98s, Train Loss: 0.681620428, Train Accuracy: 69.19%
[On Validation] ==> Val loss: 0.677938429, Total Time: 136.17s
Epoch 5/100
[On Training] ==> Time: 33.97s, Train Loss: 0.672985672, Train Accuracy: 71.72%
[On Validation] ==> Val loss: 0.666855308, Total Time: 170.14s
Epoch 6/100
[On Training] ==> Time: 33.95s, Train Loss: 0.658400188, Train Accuracy: 73.76%
[On Validation] ==> Val loss: 0.648114574, Total Time: 204.09s
Epoch 7/100
[On Training] ==> Time: 33.91s, Train Loss: 0.634059845, Tra

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task3A.mdl
[On Testing] Time: 2.637869119644165, Test Accuracy: 88.38%


Epoch 1/100
[On Training] ==> Time: 41.33s, Train Loss: 0.694334867, Train Accuracy: 50.03%
[On Validation] ==> Val loss: 0.692519313, Total Time: 41.33s
Epoch 2/100
[On Training] ==> Time: 41.28s, Train Loss: 0.691208783, Train Accuracy: 50.61%
[On Validation] ==> Val loss: 0.689922050, Total Time: 82.61s
Epoch 3/100
[On Training] ==> Time: 41.32s, Train Loss: 0.688374524, Train Accuracy: 57.25%
[On Validation] ==> Val loss: 0.686649844, Total Time: 123.93s
Epoch 4/100
[On Training] ==> Time: 41.29s, Train Loss: 0.684441089, Train Accuracy: 64.94%
[On Validation] ==> Val loss: 0.681825521, Total Time: 165.22s
Epoch 5/100
[On Training] ==> Time: 41.48s, Train Loss: 0.678350999, Train Accuracy: 69.53%
[On Validation] ==> Val loss: 0.674077965, Total Time: 206.70s
Epoch 6/100
[On Training] ==> Time: 41.33s, Train Loss: 0.668159923, Train Accuracy: 72.51%
[On Validation] ==> Val loss: 0.

In [0]:
files.download('Task3A.mdl')
files.download('Task3B.mdl')

## TASK-4

In [0]:
task4 = [(30, 20, 10, 'Task4A.mdl'), (100, 100, 0, 'Task4B.mdl'), (100, 10, 0, 'Task4C.mdl')]

In [0]:
k = 1
for num_of_hidden1, num_of_hidden2, num_of_hidden3, task_name in task4:
    if(k == 1):
        k = 2
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                
                self.drop_out1 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out2 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out3 = nn.Dropout(p=0.0) # Adding dropout layer
                
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.drop_out1(self.i2h1(x))) # relu activation @ hidden layer
                x = torch.relu(self.drop_out2(self.h12h2(x))) # relu activation @ hidden layer
                x = torch.relu(self.drop_out3(self.h22h3(x))) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
    elif(k == 2):
        k = 2
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                
                self.drop_out1 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out2 = nn.Dropout(p=0.1) # Adding dropout layer
                
                self.i2h = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h
                self.h2h = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases i2h
                self.h2o = nn.Linear(hidden_size2, output_size) # initialises weights and biases h2o

            def forward(self, x):
                x = torch.relu(self.drop_out1(self.i2h(x))) # relu activation @ hidden layer
                x = torch.relu(self.drop_out2(self.h2h(x))) # relu activation @ hidden layer
                x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
    
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 32.41s, Train Loss: 0.691956612, Train Accuracy: 57.37%
[On Validation] ==> Val loss: 0.690812181, Total Time: 32.41s
Epoch 2/100
[On Training] ==> Time: 32.61s, Train Loss: 0.688779979, Train Accuracy: 66.11%
[On Validation] ==> Val loss: 0.686643423, Total Time: 65.03s
Epoch 3/100
[On Training] ==> Time: 32.39s, Train Loss: 0.684310919, Train Accuracy: 70.87%
[On Validation] ==> Val loss: 0.681621355, Total Time: 97.42s
Epoch 4/100
[On Training] ==> Time: 32.20s, Train Loss: 0.678987091, Train Accuracy: 72.14%
[On Validation] ==> Val loss: 0.675607632, Total Time: 129.62s
Epoch 5/100
[On Training] ==> Time: 32.19s, Train Loss: 0.671137377, Train Accuracy: 74.06%
[On Validation] ==> Val loss: 0.666980039, Total Time: 161.80s
Epoch 6/100
[On Training] ==> Time: 32.64s, Train Loss: 0.659777985, Train Accuracy: 75.70%
[On Validation] ==> Val loss: 0.651878908, Total Time: 194.45s
Epoch 7/100
[On Training] ==> Time: 32.91s, Train Loss: 0.641804253, Trai

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task4A.mdl
[On Testing] Time: 3.0457141399383545, Test Accuracy: 87.68%


Epoch 1/100
[On Training] ==> Time: 32.66s, Train Loss: 0.691899299, Train Accuracy: 50.89%
[On Validation] ==> Val loss: 0.689207148, Total Time: 32.66s
Epoch 2/100
[On Training] ==> Time: 32.57s, Train Loss: 0.686522725, Train Accuracy: 60.35%
[On Validation] ==> Val loss: 0.683110623, Total Time: 65.24s
Epoch 3/100
[On Training] ==> Time: 32.80s, Train Loss: 0.678141074, Train Accuracy: 67.93%
[On Validation] ==> Val loss: 0.671698995, Total Time: 98.04s
Epoch 4/100
[On Training] ==> Time: 32.92s, Train Loss: 0.662649004, Train Accuracy: 72.03%
[On Validation] ==> Val loss: 0.650117480, Total Time: 130.96s
Epoch 5/100
[On Training] ==> Time: 33.72s, Train Loss: 0.635433326, Train Accuracy: 74.81%
[On Validation] ==> Val loss: 0.616078615, Total Time: 164.68s
Epoch 6/100
[On Training] ==> Time: 33.27s, Train Loss: 0.591664840, Train Accuracy: 77.49%
[On Validation] ==> Val loss: 0.

In [0]:
files.download('Task4A.mdl')
files.download('Task4B.mdl')
files.download('Task4C.mdl')

## Task-5

In [0]:
task5 = [('relu', 'Task5A.mdl'), ('tanh', 'Task5B.mdl'), ('sigmoid', 'Task5C.mdl')]

In [28]:
for activation, task_name in task5:
    if(activation == 'relu'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                
                self.drop_out1 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out2 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out3 = nn.Dropout(p=0.0) # Adding dropout layer
                
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.drop_out1(self.i2h1(x))) # relu activation @ hidden layer
                x = torch.relu(self.drop_out2(self.h12h2(x))) # relu activation @ hidden layer
                x = torch.relu(self.drop_out3(self.h22h3(x))) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    if(activation == 'tanh'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                
                self.drop_out1 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out2 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out3 = nn.Dropout(p=0.0) # Adding dropout layer
                
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.tanh(self.drop_out1(self.i2h1(x))) # tanh activation @ hidden layer
                x = torch.tanh(self.drop_out2(self.h12h2(x))) # tanh activation @ hidden layer
                x = torch.tanh(self.drop_out3(self.h22h3(x))) # tanh activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    if(activation == 'sigmoid'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                
                self.drop_out1 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out2 = nn.Dropout(p=0.1) # Adding dropout layer
                self.drop_out3 = nn.Dropout(p=0.0) # Adding dropout layer
                
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.sigmoid(self.drop_out1(self.i2h1(x))) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.drop_out2(self.h12h2(x))) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.drop_out3(self.h22h3(x))) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    if(activation == 'sigmoid'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.007) ## sigmoid activation converges slowly
    else:
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 31.26s, Train Loss: 0.691956612, Train Accuracy: 57.37%
[On Validation] ==> Val loss: 0.690812181, Total Time: 31.26s
Epoch 2/100
[On Training] ==> Time: 31.16s, Train Loss: 0.688779979, Train Accuracy: 66.11%
[On Validation] ==> Val loss: 0.686643423, Total Time: 62.42s
Epoch 3/100
[On Training] ==> Time: 30.68s, Train Loss: 0.684310919, Train Accuracy: 70.87%
[On Validation] ==> Val loss: 0.681621355, Total Time: 93.11s
Epoch 4/100
[On Training] ==> Time: 30.35s, Train Loss: 0.678987091, Train Accuracy: 72.14%
[On Validation] ==> Val loss: 0.675607632, Total Time: 123.45s
Epoch 5/100
[On Training] ==> Time: 31.22s, Train Loss: 0.671137377, Train Accuracy: 74.06%
[On Validation] ==> Val loss: 0.666980039, Total Time: 154.68s
Epoch 6/100
[On Training] ==> Time: 30.87s, Train Loss: 0.659777985, Train Accuracy: 75.70%
[On Validation] ==> Val loss: 0.651878908, Total Time: 185.54s
Epoch 7/100
[On Training] ==> Time: 30.40s, Train Loss: 0.641804253, Trai

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task5A.mdl
[On Testing] Time: 2.8744962215423584, Test Accuracy: 87.68%


Epoch 1/100
[On Training] ==> Time: 29.95s, Train Loss: 0.687514751, Train Accuracy: 61.52%
[On Validation] ==> Val loss: 0.682310814, Total Time: 29.95s
Epoch 2/100
[On Training] ==> Time: 31.47s, Train Loss: 0.675133548, Train Accuracy: 69.26%
[On Validation] ==> Val loss: 0.667065554, Total Time: 61.42s
Epoch 3/100
[On Training] ==> Time: 31.34s, Train Loss: 0.655745639, Train Accuracy: 72.60%
[On Validation] ==> Val loss: 0.641544085, Total Time: 92.76s
Epoch 4/100
[On Training] ==> Time: 30.99s, Train Loss: 0.622425545, Train Accuracy: 75.11%
[On Validation] ==> Val loss: 0.598505884, Total Time: 123.76s
Epoch 5/100
[On Training] ==> Time: 31.44s, Train Loss: 0.567249865, Train Accuracy: 78.38%
[On Validation] ==> Val loss: 0.534629848, Total Time: 155.20s
Epoch 6/100
[On Training] ==> Time: 30.80s, Train Loss: 0.498269229, Train Accuracy: 80.74%
[On Validation] ==> Val loss: 0.

In [0]:
files.download('Task5A.mdl')
files.download('Task5B.mdl')
files.download('Task5C.mdl')

##Task-6

In [0]:
def bigram_build_vocabulary(sentences):
    # Build bigram vocabulary
    bigrams_dictWordCount = {}
    for sent in sentences:
        for i, word in enumerate(sent.split()):
            if (i < len(sent.split())-1):
                bigrams_dictWordCount[word+' '+sent.split()[i+1]] = 0

    for sent in sentences:
        for i, word in enumerate(sent.split()):
            if (i < len(sent.split())-1):
                bigrams_dictWordCount[word+' '+sent.split()[i+1]] += 1
    max = 0
    temp = bigrams_dictWordCount.copy()
    for key, val in temp.items():
        if(bigrams_dictWordCount[key] <= 8):
            del bigrams_dictWordCount[key]

    # Mapping from index to word
    vocabulary_inv = sorted(bigrams_dictWordCount, key=bigrams_dictWordCount.__getitem__, reverse=True)
    
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return vocabulary, vocabulary_inv

In [0]:
text = []
for row in train_text_reviews+val_text_reviews+test_text_reviews:
    text.append(' '.join(row))

bigram_word_to_ix, bigram_ix_to_word = bigram_build_vocabulary(text)

In [0]:
BIGRAM_VOCAB_SIZE = len(bigram_word_to_ix)
bigram_ix_to_word[bigram_word_to_ix['worth watching']]=='worth watching', bigram_word_to_ix['worth watching'], BIGRAM_VOCAB_SIZE

(True, 82, 26519)

In [0]:
gc.collect()

1689

In [0]:
def bigram_make_bow_vector(sentence, bigram_word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(bigram_word_to_ix)).to('cuda:0') #, device=device) # make 1D vector of len = vocab size
    for word in sentence:
        if word not in bigram_word_to_ix:
#             raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            pass
        else:
            vec[bigram_word_to_ix[word]]+=1 # count the number of occurance of same word in a sentences
            
    return vec.view(1, -1)

### Prepare train bigrams

In [0]:
sentences = []
for row in train_text_reviews:
    sentences.append(' '.join(row))

train_bigram = [[word+' '+sent.split()[i+1] for i, word in enumerate(sent.split()) if i < len(sent.split())-1] for sent in sentences]

In [0]:
# store the bag of word vectors for each sentences and wraping to tensor of torch type
tic = time.time()
train_data_bigram = [Variable(bigram_make_bow_vector(instance, bigram_word_to_ix)).to('cuda:0') for instance in train_bigram]
num_train_data_bigram = len(train_bigram)
toc = time.time()
num_train_data_bigram, (toc-tic)

(16198, 25.883325338363647)

### Prepare validation bigrams

In [0]:
sentences = []
for row in val_text_reviews:
    sentences.append(' '.join(row))

val_bigram = [[word+' '+sent.split()[i+1] for i, word in enumerate(sent.split()) if i < len(sent.split())-1] for sent in sentences]

In [0]:
# store the bag of word vectors for each sentences and wraping to tensor of torch type
tic = time.time()
val_data_bigram = [Variable(bigram_make_bow_vector(instance, bigram_word_to_ix)).to('cuda:0') for instance in val_bigram]
num_val_data_bigram = len(val_bigram)
toc = time.time()
num_val_data_bigram, (toc-tic)

(1801, 2.9647693634033203)

### Prepare test bigrams

In [0]:
sentences = []
for row in test_text_reviews:
    sentences.append(' '.join(row))

test_bigrams = [[word+' '+sent.split()[i+1] for i, word in enumerate(sent.split()) if i < len(sent.split())-1] for sent in sentences]

In [0]:
# store the bag of word vectors for each sentences and wraping to tensor of torch type
tic = time.time()
test_data_bigram = [Variable(bigram_make_bow_vector(instance, bigram_word_to_ix)).to('cuda:0') for instance in test_bigrams]
num_test_data_bigram = len(test_bigrams)
toc = time.time()
num_test_data_bigram, (toc-tic)

(5000, 8.128740310668945)

In [0]:
gc.collect()

0

In [0]:
task_name = 'Task6.mdl'
class BOWClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(BOWClassifier, self).__init__()
        SEED = 42
        torch.manual_seed(SEED)
        torch.cuda.manual_seed(SEED)
        
        self.drop_out1 = nn.Dropout(p=0.1) # Adding dropout layer
        self.drop_out2 = nn.Dropout(p=0.1) # Adding dropout layer
        self.drop_out3 = nn.Dropout(p=0.0) # Adding dropout layer
        
        self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
        self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
        self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
        self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

    def forward(self, x):
        x = torch.relu(self.drop_out1(self.i2h1(x))) # relu activation @ hidden layer
        x = torch.relu(self.drop_out2(self.h12h2(x))) # relu activation @ hidden layer
        x = torch.relu(self.drop_out3(self.h22h3(x))) # relu activation @ hidden layer
        x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
        return x

num_of_input = BIGRAM_VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
num_of_hidden1 = 30
num_of_hidden2 = 20
num_of_hidden3 = 10
num_of_output = 1 # binary sentiment classes (+ve, -ve)

bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
# define a loss function and an optimizer
loss_function = nn.BCELoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.001)

gc.collect()

# Train The Model
epochs = 100
# the training loop
total_time = 0.0
prev_val_loss = float('inf')
val_loss = 0
early_stop_bow = 0

for e in range(epochs):
    tic = time.time() # start the timer
    correct = 0
    cumulative_loss = 0
    incorrect = 0

    for i, instance in enumerate(train_data_bigram): # train_text_reviews 
        # get the training data
        label = train_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        bow.zero_grad() # reset the gradient for each instance

        pred = bow.forward(instance)

        loss = loss_function(pred, label) # compute the loss
        loss.backward() # backprop the loss
        opt.step() # performs parameter updation based on the current gradient

        cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

        pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
        if(int(label) == pred_class): # counting correct prediction in each epoch
            correct += 1

    train_loss = float(cumulative_loss)/num_train_data_bigram
    train_accuracy = correct*float(100)/num_train_data_bigram

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
    val_loss = evaluate_validation_accuracy(val_data_bigram, bow) # test val-data-set on currently trained model

    if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
        bow = copy.deepcopy(early_stop_bow)
        break

    early_stop_bow = copy.deepcopy(bow)

    prev_val_loss = val_loss

    toc = time.time() # final time
    total_time += (toc-tic)

    print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
    print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

torch.save(bow, task_name)

print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
correct = 0
tic = time.time()
for i, instance in enumerate(test_data_bigram):
    label = test_text_labels[i] # get the label of the corresponding instace
    label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

    pred = bow.forward(instance)
    pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

    if(int(label) == pred_class):
        correct += 1

toc = time.time()
print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

print("\n")

Epoch 1/100
[On Training] ==> Time: 33.23s, Train Loss: 0.694220629, Train Accuracy: 50.05%
[On Validation] ==> Val loss: 0.693000334, Total Time: 33.23s
Epoch 2/100
[On Training] ==> Time: 33.16s, Train Loss: 0.692630878, Train Accuracy: 52.07%
[On Validation] ==> Val loss: 0.692187153, Total Time: 66.39s
Epoch 3/100
[On Training] ==> Time: 32.71s, Train Loss: 0.690838117, Train Accuracy: 55.86%
[On Validation] ==> Val loss: 0.689141003, Total Time: 99.09s
Epoch 4/100
[On Training] ==> Time: 32.35s, Train Loss: 0.683250456, Train Accuracy: 65.16%
[On Validation] ==> Val loss: 0.673008703, Total Time: 131.44s
Epoch 5/100
[On Training] ==> Time: 32.54s, Train Loss: 0.614814503, Train Accuracy: 74.14%
[On Validation] ==> Val loss: 0.521644103, Total Time: 163.99s
Epoch 6/100
[On Training] ==> Time: 32.96s, Train Loss: 0.393266442, Train Accuracy: 83.21%
[On Validation] ==> Val loss: 0.374230266, Total Time: 196.95s


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task6.mdl
[On Testing] Time: 3.017899751663208, Test Accuracy: 82.04%




In [0]:
files.download('Task6.mdl')

In [0]:
np.save('bigram_vocab.npy', bigram_word_to_ix)

from google.colab import files
files.download("bigram_vocab.npy")

##Task-7

In [0]:
task7 = [('ce', 'Task7A.mdl'), ('mse', 'Task7B.mdl'), ('hinge', 'Task7C.mdl')]

In [0]:
for loss_fun, task_name in task7:
    if(loss_fun == 'ce'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.BCELoss()
        
    if(loss_fun == 'mse'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.MSELoss()
        
    if(loss_fun == 'hinge'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.HingeEmbeddingLoss()
        
    # define an optimizer
    if(loss_fun == 'ce'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)
    elif(loss_fun == 'mse'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0003)
    elif(loss_fun == 'hinge'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.01)
    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 29.76s, Train Loss: 0.691906127, Train Accuracy: 58.86%
[On Validation] ==> Val loss: 0.690454737, Total Time: 29.76s
Epoch 2/100
[On Training] ==> Time: 30.48s, Train Loss: 0.688493054, Train Accuracy: 68.54%
[On Validation] ==> Val loss: 0.686511477, Total Time: 60.24s
Epoch 3/100
[On Training] ==> Time: 29.72s, Train Loss: 0.683754277, Train Accuracy: 72.19%
[On Validation] ==> Val loss: 0.680960536, Total Time: 89.96s
Epoch 4/100
[On Training] ==> Time: 29.62s, Train Loss: 0.677575701, Train Accuracy: 74.35%
[On Validation] ==> Val loss: 0.674137801, Total Time: 119.59s
Epoch 5/100
[On Training] ==> Time: 30.13s, Train Loss: 0.669296890, Train Accuracy: 76.08%
[On Validation] ==> Val loss: 0.664292754, Total Time: 149.72s
Epoch 6/100
[On Training] ==> Time: 29.71s, Train Loss: 0.656674199, Train Accuracy: 77.51%
[On Validation] ==> Val loss: 0.648659062, Total Time: 179.43s
Epoch 7/100
[On Training] ==> Time: 29.75s, Train Loss: 0.636101073, Trai

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task7A.mdl
[On Testing] Time: 2.7122726440429688, Test Accuracy: 88.10%


Epoch 1/100
[On Training] ==> Time: 29.67s, Train Loss: 0.249003833, Train Accuracy: 60.96%
[On Validation] ==> Val loss: 0.247761235, Total Time: 29.67s
Epoch 2/100
[On Training] ==> Time: 30.18s, Train Loss: 0.245982860, Train Accuracy: 71.26%
[On Validation] ==> Val loss: 0.243954085, Total Time: 59.85s
Epoch 3/100
[On Training] ==> Time: 31.28s, Train Loss: 0.241345093, Train Accuracy: 74.56%
[On Validation] ==> Val loss: 0.238403651, Total Time: 91.13s
Epoch 4/100
[On Training] ==> Time: 31.33s, Train Loss: 0.233724919, Train Accuracy: 77.00%
[On Validation] ==> Val loss: 0.228189957, Total Time: 122.46s
Epoch 5/100
[On Training] ==> Time: 30.59s, Train Loss: 0.218657703, Train Accuracy: 79.18%
[On Validation] ==> Val loss: 0.207310625, Total Time: 153.05s
Epoch 6/100
[On Training] ==> Time: 30.41s, Train Loss: 0.189368987, Train Accuracy: 81.44%
[On Validation] ==> Val loss: 0.

In [0]:
files.download('Task7A.mdl')
files.download('Task7B.mdl')
files.download('Task7C.mdl')

In [0]:
np.save('vocab.npy', word_to_ix)

from google.colab import files
files.download("vocab.npy")

In [0]:
# import matplotlib.pyplot as plt
# plt.figure("Image")
# plt.title("Loss vs Epoch")
# val_loss_history_plt =  [float(i)/sum(val_loss_history) for i in val_loss_history] # normalised between 0-1
# train_loss_history_plt =  [float(i)/sum(train_loss_history) for i in train_loss_history] # normalised between 0-1
# f_score_plt = [float(i)/sum(f_score) for i in f_score] # normalised between 0-1
# plt.plot(val_loss_history_plt, c="red", label="Validation Loss")
# plt.plot(train_loss_history_plt, c="blue", label = "Training Loss")
# plt.plot(f_score_plt, c="green", label = "F-Score")
# plt.legend()

In [0]:
# from google.colab import files
# temp_test = files.upload()

Saving Task3A.mdl to Task3A.mdl


In [0]:
# _model_ = torch.load(io.BytesIO(temp_test['Task2C.mdl']))
# _model_ = torch.load('Task5C.mdl')

In [0]:
# for params in _model_.parameters():
#     print (params)

In [0]:
# print('--- AFTER TRAINING ---')
# correct = 0
# tic = time.time()
# for i, instance in enumerate(data_test):
#     label = test_text_labels[i] # get the label of the corresponding instace
#     label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label
    
#     pred = _model_.forward(instance)
#     pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated
    
#     if(int(label) == pred_class):
#         correct += 1
# toc = time.time()
# print("Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

In [0]:
# predict on test dataset
# with open("Task1A.csv", "wb") as _f:
#     writer = csv.writer(_f)
#     writer.writerows(a)